In [ ]:
from keras._tf_keras.keras.models import Model
from keras._tf_keras.keras.layers import Input, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Dense, concatenate, Flatten, Dropout
from keras._tf_keras.keras.preprocessing.text import Tokenizer
from keras._tf_keras.keras.preprocessing.sequence import pad_sequences
from keras._tf_keras.keras.optimizers import Adam
from keras._tf_keras.keras.callbacks import EarlyStopping
from keras._tf_keras.keras.metrics import AUC, Recall, Precision

import urllib
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error

In [ ]:
file_url = "https://raw.githubusercontent.com/Matiin-heidari/hotel-review-predictor/main/data/input_data.csv"
df = pd.read_csv(file_url)
df.drop(columns=['Average_Score'], inplace=True)

In [ ]:
max_words_review = 400

## Preprocessing

In [ ]:
df['Combined Text'] = df['Review'].fillna('') + " " + df['Hotel_Address'].fillna('')
tokenizer = Tokenizer(num_words=10000, oov_token="<unk>")
tokenizer.fit_on_texts(df['Combined Text'])

text_sequences = tokenizer.texts_to_sequences(df['Combined Text'])
padded_text_sequences = pad_sequences(text_sequences, maxlen=max_words_review, padding='post', truncating='post')

vocab_size = len(tokenizer.word_index) + 1 # +1 for padding token (index 0)

In [ ]:
# Categorical Preprocessing: Hotel Name, Reviewer Nationality
le_hotel_name = LabelEncoder()
df['Hotel Name Encoded'] = le_hotel_name.fit_transform(df['Hotel_Name'])

le_reviewer_nationality = LabelEncoder()
df['Reviewer Nationality Encoded'] = le_reviewer_nationality.fit_transform(df['Reviewer_Nationality'])

In [ ]:
# Numerical Preprocessing: Hotel number reviews, Reviewer number reviews, Review Date
df['Review Month'] = pd.to_datetime(df['Review_Date'], format='%m/%d/%Y').dt.month
df['Review Year'] = pd.to_datetime(df['Review_Date'], format='%m/%d/%Y').dt.year

numerical_features_columns = ['Hotel_number_reviews', 'Reviewer_number_reviews', 'Review Month', 'Review Year']
numerical_features = df[numerical_features_columns].values

In [ ]:
X_text = padded_text_sequences
X_categorical = df[['Hotel Name Encoded', 'Reviewer Nationality Encoded']].values
X_numerical = numerical_features

In [ ]:
# Labels for outputs
y_review_type = np.array([1 if rt == 'Good_review' else 0 for rt in df['Review_Type']])
y_review_score = df['Review_Score'].values

In [ ]:
X_train_text, X_test_text, X_train_cat, X_test_cat, X_train_num, X_test_num, \
y_train_type, y_test_type, y_train_score, y_test_score = train_test_split(
    X_text, X_categorical, X_numerical, y_review_type, y_review_score,
    test_size=0.2, random_state=42, stratify=y_review_type # Stratify by review type
)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train_num)
scaled_x_train_num = scaler.transform(X_train_num)
scaled_x_test_num = scaler.transform(X_test_num)

In [ ]:
X_train = {'text_input': X_train_text, 'categorical_input': X_train_cat, 'numerical_input': scaled_x_train_num}
y_train = {'review_type_output': y_train_type, 'review_score_output': y_train_score}

X_test = {'text_input': X_test_text, 'categorical_input': X_test_cat, 'numerical_input': scaled_x_test_num}
y_test = {'review_type_output': y_test_type, 'review_score_output': y_test_score}

## Model Developement and Training

In [ ]:
def build_hotel_review_model(
    vocab_size, embedding_dim, max_sequence_length,
    num_categorical_features, num_numerical_features,
    lstm_units=128, dense_units_text=64, dense_units_categorical=32,
    dense_units_numerical=32, shared_dense_layers=2,
    shared_dense_units=128, dropout_rate=0.3
):
    text_input = Input(shape=(max_sequence_length,), name='text_input')
    x_text = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length)(text_input)

    x_text = LSTM(lstm_units, return_sequences=False)(x_text)

    x_text = Dense(dense_units_text, activation='relu')(x_text)
    x_text = Dropout(dropout_rate)(x_text)


    categorical_input = Input(shape=(num_categorical_features,), name='categorical_input')

    x_cat = Dense(dense_units_categorical, activation='relu')(categorical_input)
    x_cat = Dropout(dropout_rate)(x_cat)

    numerical_input = Input(shape=(num_numerical_features,), name='numerical_input')
    x_num = Dense(dense_units_numerical, activation='relu')(numerical_input)
    x_num = Dropout(dropout_rate)(x_num)

    # Merging All Branches
    merged = concatenate([x_text, x_cat, x_num])

    # Shared Dense Layers
    shared_output = merged
    for _ in range(shared_dense_layers):
        shared_output = Dense(shared_dense_units, activation='relu')(shared_output)
        shared_output = Dropout(dropout_rate)(shared_output)

    # Output Layers

    # Classification Head: Predicts Review Type (Bad/Good)
    classification_output = Dense(1, activation='sigmoid', name='review_type_output')(shared_output)

    # Regression Head: Predicts Review Score
    regression_output = Dense(1, activation='linear', name='review_score_output')(shared_output)

    # Define the model with multiple inputs and multiple outputs
    model = Model(
        inputs=[text_input, categorical_input, numerical_input],
        outputs=[classification_output, regression_output]
    )
    return model

In [ ]:
# Hyperparameters:
embedding_dim_hparam = 100
lstm_units_hparam = 128
dense_units_text_hparam = 64
dense_units_categorical_hparam = 32
dense_units_numerical_hparam = 32
shared_dense_units_hparam = 128
dropout_rate_hparam = 0.3
learning_rate_hparam = 0.001
batch_size_hparam = 32
epochs_hparam = 50
loss_weights_hparam = {'review_type_output': 0.5, 'review_score_output': 0.5}

In [ ]:
# Initialize the model
model = build_hotel_review_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim_hparam,
    max_sequence_length=max_words_review,
    num_categorical_features=X_categorical.shape[1],
    num_numerical_features=X_numerical.shape[1],
    lstm_units=lstm_units_hparam,
    dense_units_text=dense_units_text_hparam,
    dense_units_categorical=dense_units_categorical_hparam,
    dense_units_numerical=dense_units_numerical_hparam,
    shared_dense_layers=2,
    shared_dense_units=shared_dense_units_hparam,
    dropout_rate=dropout_rate_hparam
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(
    optimizer=Adam(learning_rate=learning_rate_hparam),
    loss={
        'review_type_output': 'binary_crossentropy',
        'review_score_output': 'mean_squared_error'
    },
    loss_weights=loss_weights_hparam
)


In [ ]:
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 400)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_6         │ (None, 400, 100)  │  1,393,200 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_6 (LSTM)       │ (None, 128)       │    117,248 │ embedding_6[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ categorical_input   │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numerical_input     │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_30 (Dense)    │ (None, 64)        │      8,256 │ lstm_6[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_31 (Dense)    │ (None, 32)        │         96 │ categorical_inpu… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_32 (Dense)    │ (None, 32)        │        160 │ numerical_input[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_30          │ (None, 64)        │          0 │ dense_30[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_31          │ (None, 32)        │          0 │ dense_31[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_32          │ (None, 32)        │          0 │ dense_32[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, 128)       │          0 │ dropout_30[0][0], │
│ (Concatenate)       │                   │            │ dropout_31[0][0], │
│                     │                   │            │ dropout_32[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_33 (Dense)    │ (None, 128)       │     16,512 │ concatenate_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_33          │ (None, 128)       │          0 │ dense_33[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_34 (Dense)    │ (None, 128)       │     16,512 │ dropout_33[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_34          │ (None, 128)       │          0 │ dense_34[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ review_type_output  │ (None, 1)         │        129 │ dropout_34[0][0]  │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ review_score_output │ (None, 1)         │        129 │ dropout_34[0][0]  │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴─────────────────

 Total params: 1,552,242 (5.92 MB)

 Trainable params: 1,552,242 (5.92 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=epochs_hparam,
    batch_size=batch_size_hparam,
    validation_split=0.2,
    callbacks=[early_stopping_callback],
    verbose=1
)

Epoch 1/50
276/276 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - loss: 750.9824 - review_score_output_loss: 1487.9196 - review_type_output_loss: 14.0372 - val_loss: 5.8378 - val_review_score_output_loss: 10.6161 - val_review_type_output_loss: 1.0638
Epoch 2/50
276/276 ━━━━━━━━━━━━━━━━━━━━ 7s 25ms/step - loss: 25.3085 - review_score_output_loss: 48.0514 - review_type_output_loss: 2.5654 - val_loss: 8.0319 - val_review_score_output_loss: 15.3319 - val_review_type_output_loss: 0.7372
Epoch 3/50
276/276 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - loss: 9.3796 - review_score_output_loss: 17.6135 - review_type_output_loss: 1.1456 - val_loss: 6.4892 - val_review_score_output_loss: 12.2601 - val_review_type_output_loss: 0.7240
Epoch 4/50
276/276 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 5.9084 - review_score_output_loss: 11.0183 - review_type_output_loss: 0.7984 - val_loss: 4.9469 - val_review_score_output_loss: 9.2044 - val_review_type_output_loss: 0.6942
Epoch 5/50
276/276 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/ste

## Model Evaluation on Unseen Test Data

In [ ]:
test_loss_results = model.evaluate(X_test, y_test, verbose=1)


87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 3.2027 - review_score_output_loss: 5.6919 - review_type_output_loss: 0.7136


In [ ]:
predictions = model.predict(X_test)
y_pred_type_probs = predictions[0]
y_pred_score = predictions[1]

y_pred_type_binary = (y_pred_type_probs > 0.5).astype(int)

87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


### Review Type Prediction Metrics:

In [ ]:
# Calculate Accuracy
accuracy = accuracy_score(y_test_type, y_pred_type_binary)
print(f"Accuracy: {accuracy:.4f}")

# Calculate Precision
precision = Precision()(y_test_type, y_pred_type_probs).numpy()
print(f"Precision: {precision:.4f}")

# Calculate Recall
recall = Recall()(y_test_type, y_pred_type_probs).numpy()
print(f"Recall: {recall:.4f}")

# Calculate F1 Score
f1 = f1_score(y_test_type, y_pred_type_binary)
print(f"F1 Score: {f1:.4f}")

# Calculate ROC AUC
roc_auc = roc_auc_score(y_test_type, y_pred_type_probs)
print(f"ROC AUC: {roc_auc:.4f}")

Accuracy: 0.5002
Precision: 0.5000
Recall: 1.0000
F1 Score: 0.6667
ROC AUC: 0.4648


### Review Score Prediction Metrics:

In [ ]:
# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test_score, y_pred_score)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test_score, y_pred_score)
print(f"Mean Squared Error (MSE): {mse:.4f}")

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")


Mean Absolute Error (MAE): 1.9721
Mean Squared Error (MSE): 5.7464
Root Mean Squared Error (RMSE): 2.3972
